In [5]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"


import numpy as np
import cv2
from glob import glob
from tqdm import tqdm
import tensorflow as tf

#read images
images = glob("C:/Users/Du/Documents/ref-matte/train/mask/*")
#images = glob("../imgs/*")
print(f"Images: {len(images)}")

if not os.path.exists("results/"):
    os.makedirs("results")

#separate images by name and store in names dictionary
dic = dict()
print("resolving names...")
for x in tqdm(images, total = len(images)):
        """ Extracting the name """
        name = x.split("\\")[-1] #windows format
        nameVec = []
        nameVec.append(x)
        nameKey = name.split("_")[2]
        if nameKey not in dic:
            dic[nameKey] = []
        dic[nameKey].append(nameVec[0])

#for each key read images and put in images dic
image = dict()
print("resolving images...")
for key in tqdm(dic, total = len(dic)):
      image[key] = []
      for value in dic[key]:    
          image[key].append(cv2.imread(value, cv2.IMREAD_COLOR))

#for each key merge images inside and save in path

imagelist = [[0 for x in range(3)] for y in range(len(image.keys()))]
print("len = ", len(image.keys()))
i = 0
print("blending images...")
for key in tqdm(image, total = len(image)):
    for j in range(len(image[key])):
        print("i = ", i)
        print("j = ", j)
        if j == 0:
            imagelist[i] = image[key][j]
        else:
            imagelist[i] = cv2.addWeighted(imagelist[i], 1.0, image[key][j], 1.0, 0)
    name = "".join(["rim_train_", key,".jpg"]) 
    save_image_path = os.path.join("results", name)
    cv2.imwrite(save_image_path, imagelist[i])
    i += 1


Images: 42
resolving names...


100%|██████████| 42/42 [00:00<?, ?it/s]


resolving images...


100%|██████████| 18/18 [00:01<00:00, 12.12it/s]


len =  18
blending images...


...

i =  9
j =  0
i =  9
j =  1
i =  9
j =  2
i =  10
j =  0
i =  10
j =  1
i =  11
j =  0
i =  11
j =  1
i =  11
j =  2


 78%|███████▊  | 14/18 [00:01<00:00, 13.03it/s]

i =  12
j =  0
i =  12
j =  1
i =  12
j =  2
i =  13
j =  0
i =  13
j =  1
i =  14
j =  0
i =  14
j =  1
i =  15
j =  0
i =  15
j =  1


100%|██████████| 18/18 [00:01<00:00, 13.76it/s]

i =  16
j =  0
i =  16
j =  1
i =  17
j =  0
i =  17
j =  1
i =  17
j =  2
